Himani Parikh - 1322085

CSCI 636 - Big Data Analytics

Hands on 10.1 : Spark-Classification using decision decision tree(low-humidity days vs. non-low- humidity days at 9am)

***Install pyspark***

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=03d5a9e2ee7c97a78c886eab1646eb207b603caeefe065ede24841519cbf0513
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

***Mount Google Drive***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change the directory to big-data-colab folder
%cd /content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh

#pwd
!ls

/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh
 10.Spark-ClassificationHimani.ipynb	        out
 2.DataCopyHimani.ipynb			        out_Datajoin
 2_WordCountHimani.ipynb		        outDatajoin2
 3.1_WordGrepHimani.ipynb		        out_grep
 3.2_WordMedianHimani.ipynb		        out_testfile
 4.MR-DataJoinHimani.ipynb		        out_testfile1
 4.MR-WordCountHimani.ipynb		        out_testfile2
 5.MR-DataJoin2Himani.ipynb		        out_wordmedian
 6.MR-WordCountReducerHimani.ipynb	        SparkOutput
 7.Spark-WordCountHimani.ipynb		        SqlDataSet
 8.Spark-SQLHimani.ipynb		        testfile1
 8.Spark-StreamingHimani.ipynb		        testfile2
 9.Spark-Handling-missing-values-Himani.ipynb   WeatherDataset
 BigDataHimani.ipynb			        wordcount_mapper.py
 Data					        wordcount_num0_output.txt
 Data_join				        wordcount_num2_output.txt
 Data_join2				        wordcount_reducer.py
'Hands On 9.2-Himani.ipynb'


***Load classes and data***

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer

In [ ]:
sqlsc = SQLContext(sc)

df = sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/WeatherDataset/daily_weather.csv',
                     format='com.databricks.spark.csv',
                     header='true',
                     inferSchema='true')

df.columns

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [ ]:
featureColumns = ['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am']

***Drop unused and missing data***

In [ ]:
df = df.drop('number')

In [ ]:
df = df.na.drop()
df.count(), len(df.columns)

(1064, 10)

***Create categorical variable***

In [ ]:
binarizer = Binarizer(threshold=24.99999, inputCol='relative_humidity_3pm', outputCol='label')
binarizerDF = binarizer.transform(df)

In [ ]:
binarizerDF.select('relative_humidity_3pm', 'label').show(4)

+---------------------+-----+
|relative_humidity_3pm|label|
+---------------------+-----+
|   36.160000000000494|  1.0|
|     19.4265967985621|  0.0|
|   14.460000000000045|  0.0|
|   12.742547353761848|  0.0|
+---------------------+-----+
only showing top 4 rows



***Aggregate features***

In [ ]:
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
assembled = assembler.transform(binarizerDF)

***Split training and test data***

In [ ]:
(trainingData, testData) = assembled.randomSplit([0.8, 0.2], seed=13234)

In [ ]:
trainingData.count(), testData.count()

(846, 218)

***Create and train decision tree***

In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=5,
                       minInstancesPerNode=20, impurity="gini")

In [ ]:
pipeline = Pipeline(stages=[dt])
model = pipeline.fit(trainingData)

In [ ]:
predictions = model.transform(testData)

In [ ]:
predictions.select("prediction", "label").show(10)

+----------+-----+
|prediction|label|
+----------+-----+
|       1.0|  1.0|
|       1.0|  1.0|
|       0.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       0.0|  0.0|
|       1.0|  1.0|
+----------+-----+
only showing top 10 rows



***Save predicitions to CSV***

In [ ]:
predictions.select("prediction", "label").write.save(path="/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/Prediction/predictions.csv",
                                                     format='com.databricks.spark.csv',
                                                     header='true')